<h2><center>Exploring the World of Language Models:</center></h2>
<h1><center>LLMs vs Chat Models</center></h1>
<hr><hr><hr>
<h3>Introduction</h3>
Large Language Models have made significant advancements in the field of Natural Language Processing (NLP), enabling AI systems to understand and generate human-like text. ChatGPT is a popular language model based on Transformers architecture, enabling it to understand long texts and figure out how words or ideas are connected. It's great at making predictions about language and relationships between words.

<h3>LLMs v/s Chat Models</h3>
LLMs and Chat Models are two types of models in LangChain, serving different purposes in natural language processing tasks. This lesson will examine the differences between LLMs and Chat Models, their unique use cases, and how they are implemented within LangChain. 

# Summary:
------------
- Classes `OpenAI` and `AzureOpenAI` are wrapper of **LLM models**.
- Classes `ChatOpenAI` and `AzureChatOpenAI`are wrapper of **Chat Models**
------------------------------------------------------------------------------------------------------------------------------------------------------------------
- LLM models take **string value** as parameter.
    - Example-> `AzureOpenAI( prompt: str )` , `OpenAI( prompt: str )`
- Chat models take **string** or **List of Base Messages(e.g.: `SystemMessage`, `HumanMessage`, `AIMessage`)** or **`PromptValue`(output of `PromptTemplate` or `ChatPromptTemplate`) object** as parameter.
    - Example-> `AzureChatOpenAI( prompt: List[BaseMessages] or PromptValue or str )` , `ChatOpenAI( prompt: List[BaseMessages] or PromptValue or str )`
-----------------------------------------------------------------------------------------------------------------------------------------------------------------
- To provide **`PromptTemplate` or `ChatPromptTemplate` object** as parameter to **Chat Models** `.invoke()` method directly, we need to use `.format_messages()` method of the `PromptTemplate` or `ChatPromptTemplate` object to convert it to a `PromptValue` object. Then, it can be passed as parameter to **Chat Models** `.invoke()` method directly.
    - Example:<br>
      `chat_model = AzureChatOpenAI(...)` <br>
      `prompt = ChatPromptTemplate.from_messages(....) or ChatPromptTemplate.from_template(....)` <br>
      `response = chat_model( prompt.format_messages(...) )`
- When prompt and Chat Model are used in an LLM chain, then prompt is automatically converted internally to messages using `.format_messages()`, and then fed to Chat Model for getting response.
------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Configurations:-
---------------------

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_api_version = "2023-05-15"
llm_deployment_name = os.getenv("GPT_DEPLOYMENT_NAME")

os.environ["OPENAI_API_TYPE"]     = "azure"
os.environ["OPENAI_API_VERSION"]  = azure_openai_api_version
os.environ["OPENAI_API_KEY"]      = azure_openai_api_key

# LLMs:
--------
LLMs, such as GPT-3, Bloom, PaLM, and Aurora genAI, take a text string as input and return a text string as output. They are trained on language modeling tasks and can generate human-like text, perform complex reasoning, and even write code. LLMs are powerful and flexible, capable of generating text for a wide range of tasks. **However, they can sometimes produce incorrect or nonsensical answers, and their API is less structured compared to Chat Models.**

Pre-training these models involves presenting large-scale corpora to them and allowing the network to predict the next word, which results in learning the relationships between words. This learning process enables LLMs to generate high-quality text, which can be applied to an array of applications, such as automatic form-filling and predictive text on smartphones.

Most of these models are trained on general purpose training dataset, while others are trained on a mix of general and domain-specific data, such as Intel Aurora genAI, which is trained on general text, scientific texts, scientific data, and codes related to the domain. The goal of domain specific LLMs is to increase the performance on a particularly domain, while still being able to solve the majority of tasks that general LLM can manage.

LLMs have the potential to infiltrate various aspects of human life, including the arts, sciences, and law. With continued development, LLMs will become increasingly integrated into our educational, personal, and professional lives, making them an essential technology to master.

You can follow these steps to use a large language model (LLM) like GPT-3 in LangChain. Import the `OpenAI` wrapper from the `langchain.llms` module and Initialize it with the desired model name and any additional arguments. For example, set a high temperature for more random outputs. Then, create a `PromptTemplate` to format the input for the model. Lastly, define an `LLMChain` to combine the model and prompt. Run the chain with the desired input using `.invoke()`.

In [3]:
from langchain_openai import AzureOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [4]:
llm = AzureOpenAI(
    deployment_name=llm_deployment_name,
    api_version=azure_openai_api_version
)

In [7]:
response = llm.invoke("Tips of packing personal stuffs before going on a long vacation abroad.")
print( response.content )

BadRequestError: Error code: 400 - {'error': {'code': 'OperationNotSupported', 'message': 'The completion operation does not work with the specified model, gpt-35-turbo. Please choose different model and try again. You can learn more about which models can be used with each operation here: https://go.microsoft.com/fwlink/?linkid=2197993.'}}

In [8]:
prompt = PromptTemplate(
  input_variables=["product"],
  template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

print( chain.invoke("wireless headphones") )

BadRequestError: Error code: 400 - {'error': {'code': 'OperationNotSupported', 'message': 'The completion operation does not work with the specified model, gpt-35-turbo. Please choose different model and try again. You can learn more about which models can be used with each operation here: https://go.microsoft.com/fwlink/?linkid=2197993.'}}

The above two code snippets generates the following error: <br>
`
BadRequestError: Error code: 400 - {'error': {'code': 'OperationNotSupported', 'message': 'The completion operation does not work with the specified model, gpt-35-turbo. Please choose different model and try again. You can learn more about which models can be used with each operation here: https://go.microsoft.com/fwlink/?linkid=2197993.'}}
`

The reason for the error is as below:
Langchain version used in this notebook code: 0.1.9
<img src="./images/04-AzureOpenAI chat completion error.jpg" />

# Chat Models:
----------------
- Chat Models are the most popular models in LangChain, such as ChatGPT that can incorporate GPT-3 or GPT-4 at its core. They have gained significant attention due to their ability to learn from human feedback and their user-friendly chat interface.
- Chat Models, such as ChatGPT, take a list of messages as input and return an `AIMessage`.
- Chat Models typically use LLMs as their underlying technology, but **their APIs are more structured**.
- Chat Models are designed to remember previous exchanges with the user in a session and use that context to generate more relevant responses.
- They also benefit from **reinforcement learning** from human feedback, which helps improve their responses.
- However, Chat Models may still have limitations in reasoning and require careful handling to avoid generating inappropriate content.

### Chat Message Types:
In LangChain, three main types of messages are used when interacting with chat models: `SystemMessage`, `HumanMessage`, and `AIMessage`.

- `SystemMessage`: These messages provide initial instructions, context, or data for the AI model. They set the objectives the AI should follow and can help in controlling the AI's behavior. **It gives the AI model a *Persona*, to follow for that entire chat specifically**. System messages are not user inputs but rather guidelines for the AI to operate within.

- `HumanMessage`: These messages come from the user and represent their input to the AI model. **The AI model is expected to respond to these messages.** In LangChain, you can customize the human prefix (e.g., "User") in the conversation summary to change how the human input is represented.

- `AIMessage`: These messages are sent from the AI's perspective as it interacts with the human user. **They represent the AI's responses to human input.** Like HumanMessage, you can also customize the AI prefix (e.g., "AI Assistant" or "AI") in the conversation summary to change how the AI's responses are represented.

### An example of using `AzureChatOpenAI` with a `HumanMessage`: 
------------------------------------------------------------------
In this section, we are trying to use the LangChain library to create a chatbot that can translate an English sentence into Bengali. This particular use case goes beyond what we covered in the previous lesson. <u>We'll be employing multiple message types to differentiate between users' queries and system instructions instead of relying on a single prompt. This approach will enhance the model's comprehension of the given requirements.</u>

First, we create a list of messages, starting with a `SystemMessage` that sets the context for the chatbot, informing it that its role is to be a helpful assistant translating English to Bengali. We then follow it with a `HumanMessage` containing the user’s query, like an English sentence to be translated.

In [3]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import (
  HumanMessage,
  SystemMessage,
  AIMessage
)

chat = AzureChatOpenAI(
    openai_api_version = azure_openai_api_version,
    azure_deployment = llm_deployment_name,
    temperature = 0.9
)

### Coding Approach 1: Using list of langchain message objects as parameter to Chat Model `.invoke()` method:
---------------------------------------------------------------------------------------------------------------

In [4]:
messages = [
	SystemMessage(content="You are a helpful assistant that translates English to Bengali."),
	HumanMessage(content="Translate the following sentence: I love programming.")
]

print( chat.invoke(messages) )

content='আমি প্রোগ্রামিং ভালবাসি।'


### Coding Approach 2: Converting `ChatPromptTemplate` object to messages using `.format_messages()` method, and using its result as parameter to Chat Model `.invoke()` method:
---------------------------------------------------------------------------------------------------------
- <a href="https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html" target="_blank">Know more about ChatPromptTemplate</a>

In [8]:
chat_prompt = ChatPromptTemplate.from_messages([
	("system", "You are a helpful assistant that translates English to Bengali. Your task is to translate the entire sentence provided below by the user into Bengali language."),
	("user", "{user_input}")
])

print( chat.invoke( chat_prompt.format_messages( user_input="I want to live freely and happily!" ) ) )

content='আমি স্বাধীনভাবে এবং সুখে জীবন কাটাতে চাই! (Ami swadhinhabe ebong sukhe jibon kataate chai!)'


### Coding Approach 3: Using `ChatPromptTemplate` object as parameter LLM Chain `.invoke()` method:
---------------------------------------------------------------------------------------------------------
- <a href="https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html" target="_blank">Know more about ChatPromptTemplate</a>

In [10]:
chat_prompt = ChatPromptTemplate.from_messages([
	("system", "You are a helpful assistant that translates English to Bengali. Your task is to translate the entire sentence provided below by the user into Bengali language."),
	("user", "{user_input}")
])

chat_chain = chat_prompt | chat

In [11]:
user_input = "The weather today is very beautiful."

response = chat_chain.invoke( {"user_input": user_input} )
print( response )

content='আজকের আবহাওয়া খুব সুন্দর। (Aajker abhawaoya khub sundor.)'


# Chat Generation:
## Using `.generate()` method to pass multiple prompts and get multiple independent responses simultaneously:-
------------------------------------------------------------------------------------------------------------------
- Using the `llm.generate()` method, you can also generate completions for multiple sets of messages.
- Each batch of messages can have its own `SystemMessage` and will perform independently.
- The following code shows the first set of messages translate the sentences from English to Bengali, while the second ones do the opposite.

In [15]:
batch_messages = [
  [
    SystemMessage(content="You are a helpful assistant that translates English to Bengali."),
    HumanMessage(content="Translate the following sentence: I love programming.")
  ],
  [
    SystemMessage(content="You are a helpful assistant that translates Bengali to English."),
    HumanMessage(content="Translate the following sentence: আজকের আবহাওয়া খুব সুন্দর।")
  ],
]


response = chat.generate(batch_messages)
print( response )

generations=[[ChatGeneration(text='আমি প্রোগ্রামিং ভালবাসি। (Ami programming bhalobashi.)', generation_info={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, message=AIMessage(content='আমি প্রোগ্রামিং ভালবাসি। (Ami programming bhalobashi.)'))], [ChatGeneration(text="Today's weather is very beautiful.", generation_info={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, message=AIMessage(content="Today's weather is very beautiful."))]] llm_output={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 97, 'total_tokens': 143}, 'model_name': 'gpt-3.5-turbo'} run=[RunInfo(run_id=UUID('c8d14e8f-bce8-4024-8b5c-0bcbbce6ca46')), RunInfo(run_id=UUID('77b48b96-2d39-4634-9726-be01460c232b'))]


In [19]:
for obj in response.generations:
    print( obj, "\n\n" )

[ChatGeneration(text='আমি প্রোগ্রামিং ভালবাসি। (Ami programming bhalobashi.)', generation_info={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, message=AIMessage(content='আমি প্রোগ্রামিং ভালবাসি। (Ami programming bhalobashi.)'))] 


[ChatGeneration(text="Today's weather is very beautiful.", generation_info={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, message=AIMessage(content="Today's weather is very beautiful."))] 




- The above response contains an object that has a list of `ChatGeneration` objects, which content the actual completions for the user query as `text` attribute, as well as in the form of an `AIMessage` object. 

In [20]:
for obj in response.generations:
    print( obj[0].text, "\n\n" )

আমি প্রোগ্রামিং ভালবাসি। (Ami programming bhalobashi.) 


Today's weather is very beautiful. 


